# PhoneLM

## Test `G2P` and `Encodec`

In [ ]:
!pip install g2p_en encodec

### `G2P`

In [2]:
from g2p_en import G2p

In [3]:
import torch
import random
import string
from functools import cache
from tqdm import tqdm

@cache
def _get_model():
    return G2p()

@cache
def _get_graphs(path):
    with open(path, "r") as f:
        graphs = f.read()
    return graphs

def encode(graphs: str) -> list[str]:
    g2p = _get_model()
    phones = g2p(graphs)
    ignored = {" ", *string.punctuation}
    return ["_" if p in ignored else p for p in phones]

@torch.no_grad()
def write_phones(folder, suffix=".normalized.txt"):
    print("ello?")
    paths = list(folder.rglob(f"*{suffix}"))
    random.shuffle(paths)

    print("paths:", paths)
    for path in tqdm(paths):
        phone_path = path.with_name(path.stem.split(".")[0] + ".phn.txt")
        if phone_path.exists():
            continue
        print("?")
        graphs = _get_graphs(path)
        phones = encode(graphs)
        with open(phone_path, "w") as f:
            f.write(" ".join(phones))

In [4]:
from pathlib import Path
write_phones(Path("./data/text"))

ello?
paths: [WindowsPath('data/text/test.normalized.txt')]


100%|██████████| 1/1 [00:00<?, ?it/s]


### `Encodec`

In [36]:
from tqdm import tqdm
import random
import torch
from functools import cache
import torchaudio
from encodec import EncodecModel
from torch import Tensor
from einops import rearrange
import soundfile
from encodec.utils import convert_audio

SAMPLE_RATE = 24_000
BANDWIDTHS  = [1.5, 3.0, 6.0, 12.0, 24.0]
BANDWIDTH   = BANDWIDTHS[0]

@cache
def _load_model(bandwidth=6.0, device="cuda"):
    # Instantiate a pretrained EnCodec model
    assert SAMPLE_RATE == 24_000
    model = EncodecModel.encodec_model_24khz()
    model.set_target_bandwidth(bandwidth)
    model.to(device)
    return model

def unload_model():
    return _load_model.cache_clear()

@torch.inference_mode()
def decode(codes: Tensor, bandwidth=6.0, device="cuda"):
    """
    Args:
        codes: (b q t)
    """
    assert codes.dim() == 3
    model = _load_model(bandwidth, device)
    return model.decode([(codes, None)]), model.sample_rate

def decode_to_file(resps: Tensor, path: Path):
    assert resps.dim() == 2, f"Require shape (t q), but got {resps.shape}."
    resps = rearrange(resps, "t q -> 1 q t")
    wavs, sr = decode(codes=resps, bandwidth=BANDWIDTH)
    soundfile.write(str(path), wavs.cpu()[0, 0], sr)

def _replace_file_extension(path, suffix):
    return (path.parent / path.name.split(".")[0]).with_suffix(suffix)

@torch.inference_mode()
def encode(wav: Tensor, sr: int, bandwidth=6.0, device="cuda"):
    """
    Args:
        wav: (t)
        sr: int
    """
    model = _load_model(bandwidth, device)
    wav = wav.unsqueeze(0)
    wav = convert_audio(wav, sr, model.sample_rate, model.channels)
    wav = wav.to(device)
    encoded_frames = model.encode(wav)
    qnt = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1)  # (b q t)
    return qnt

def encode_from_file(path, bandwidth=6.0, device="cuda"):
    wav, sr = torchaudio.load(str(path))
    if wav.shape[0] == 2:
        wav = wav[:1]
    return encode(wav, sr, bandwidth, device)

def quantize_audio(folder, suffix=".wav"):
    paths = [*folder.rglob(f"*{suffix}")]
    random.shuffle(paths)

    for path in tqdm(paths):
        out_path = _replace_file_extension(path, ".qnt.pt")
        if out_path.exists():
            continue
        qnt = encode_from_file(path, BANDWIDTH)
        print(qnt.shape)
        torch.save(qnt.cpu(), out_path)

def decode_files(folder, suffix=".qnt.pt"):
    paths = [*folder.rglob(f"*{suffix}")]
    random.shuffle(paths)

    for path in tqdm(paths):
        out_path = _replace_file_extension(path, ".qt.wav")
        if out_path.exists():
            continue
        fi = rearrange(torch.load(path).squeeze(0).cuda(), "q t -> t q")
        decode_to_file(fi, out_path)

In [37]:
from pathlib import Path
quantize_audio(Path("./data/audio"))
decode_files(Path("./data/audio"))

100%|██████████| 1/1 [00:00<00:00,  2.86it/s]


torch.Size([1, 2, 128])


100%|██████████| 1/1 [00:00<00:00, 58.82it/s]


In [27]:
torch.load("data/audio/test.qnt.pt").shape

torch.Size([1, 2, 128])

## Dataset

### LJSpeech

In [200]:
BANDWIDTH_IDX = 1 # original VALL-E
CODEBOOKS     = [2, 4, 8, 16, 32]
BANDWIDTHS    = [1.5, 3.0, 6.0, 12.0, 24.0]
BANDWIDTH     = BANDWIDTHS[BANDWIDTH_IDX]
CODEBOOK      = CODEBOOKS[BANDWIDTH_IDX]

import torchaudio
from ljspeech import LJSPEECH
DATASET_PATH = "./data/LJSpeech/"
dataset = LJSPEECH(
    "./data/LJSpeech",
    encodec_bandwidth=BANDWIDTH)

In [201]:
len(dataset)

13100

In [202]:
dataset[0][-1].shape

torch.Size([1, 4, 725])

In [203]:
import torch
import torchaudio
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.model_selection import train_test_split

indices = list(range(len(dataset)))
train_indices, test_indices = train_test_split(indices, test_size=0.1, random_state=42)

train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = DataLoader(dataset, batch_size=32, sampler=train_sampler, collate_fn=lambda x: x)
test_loader = DataLoader(dataset, batch_size=32, sampler=test_sampler, collate_fn=lambda x: x)

In [204]:
len(train_loader), len(test_loader)

(369, 41)

In [205]:
item = next(iter(train_loader))

In [ ]:
item

## Model

In [207]:
import megabyte
import torch
import torch.nn as nn
from einops import rearrange

def get_reserved_mem_gb():
    device = torch.cuda.current_device()
    reserved = torch.cuda.memory_reserved(device)
    reserved_gb = reserved / 1024 / 1024 / 1024
    return reserved_gb

class PhoneLM(nn.Module):
    def __init__(self, n_phone_tokens, n_audio_tokens):
        super(PhoneLM, self).__init__()
        self.megabyte   = megabyte.MEGABYTE(
            heads       = 8, # 1,
            dim_head    = 32, # 16,
            num_tokens  = n_phone_tokens + n_audio_tokens + 4,
            dim         = (768, 256, 128), # (32, 32, 32), # (768, 256, 128)# Dg, Dl1, Dl2
            depth       = (6, 4, 2), # (6, 4, 2)
            max_seq_len = (128, 4, 4), # (32, 4, 4), # 512
            flash_attn  = False)

    def forward(self, x, debug=False, return_loss=True):
        x = self.megabyte(x, return_loss=return_loss)
        return x
    
    def get_params(self):
        o = [param.numel() for param in self.parameters() if param.requires_grad]
        o = sum(o)
        return o
    
    def generate(self, *args):
        return self.megabyte.generate(*args)
    
def multi_encode(
        phone_tokens,
        audio_tokens,
        n_phone_tokens,
        n_audio_tokens,
        max_clip_length=1.0):
    """NOTE: 75 steps per second for 24kHz in `encodec.
    Set `max_clip_length` to 0 for original clip length."""

    # Start text token, end text token, start audio token, end audio token
    STT, ETT, SAT, EAT = [n_phone_tokens + n_audio_tokens + i
                          for i in range(4)]
    STT = torch.tensor([STT]).long().cuda()
    ETT = torch.tensor([ETT]).long().cuda()
    SAT = torch.tensor([SAT]).long().cuda()
    EAT = torch.tensor([EAT]).long().cuda()

    if max_clip_length:
        print("pre audio_tokens.shape", audio_tokens.shape)
        audio_tokens = audio_tokens[:, :, :int(max_clip_length * 75)]
    audio_tokens = rearrange(audio_tokens.squeeze(0), "q s -> (q s)")
    print("post audio_tokens.shape", audio_tokens.shape)
    
    # offset phone tokens past audio tokens
    phone_tokens += n_audio_tokens
    
    print("phone_tokens.shape:", phone_tokens.shape)
    print("audio_tokens.shape:", audio_tokens.shape)
    
    device = torch.cuda.current_device()
    phone_tokens = torch.cat((STT, phone_tokens, ETT), dim=0).to(device)
    audio_tokens = torch.cat((SAT, audio_tokens, EAT,), dim=0).to(device)
    combined_tokens = torch.cat((phone_tokens, audio_tokens), dim=0).to(device)
    return phone_tokens, audio_tokens, combined_tokens

In [208]:
from einops import rearrange

from encodec_util import decode_to_file

def generate_audio(sample,
                   n_phone_tokens,
                   n_audio_tokens,
                   audio_path="./out.wav"):
    STT, ETT, SAT, EAT = [n_phone_tokens + n_audio_tokens + i
                          for i in range(4)]
    ST_S = [STT, ETT, SAT, EAT]
    print("STT, ETT, SAT, EAT ids:", ST_S)
    seq = sample.cpu().tolist()[0]
    print("seq:", seq)
    # all special tokens in list
    if all(st_t in seq for st_t in ST_S) and len(seq) >= len(ST_S) + 2:
        # text_tokens  = seq[seq.index(STT + 1):seq.index(ETT - 1)]
        audio_tokens = seq[seq.index(SAT)+1:seq.index(EAT)]
        print(seq.index(SAT), seq.index(EAT), len(audio_tokens))
        audio_tokens = torch.tensor(audio_tokens).cuda()
        audio_tokens = rearrange(
            audio_tokens,
            '(t q) -> t q',
            q=CODEBOOK,
            t=audio_tokens.size(0) // CODEBOOK)
        print("audio_tokens.shape:", audio_tokens, audio_tokens.shape)
        decode_to_file(audio_tokens, audio_path)
        return True
    else:
        return False

## PhoneLM - LJSpeech

### Train

In [209]:
device  = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PhoneLM(
    n_phone_tokens=len(dataset.phone_dict),
    n_audio_tokens=1024).to(device)

model.megabyte.get_num_params()

number of parameters: 37.30M


37302863

In [210]:
item = next(iter(train_loader))[0]
item_phone_tokens = item[-2]
item_audio_tokens = item[-1]
item_phone_tokens.shape, item_audio_tokens.shape

(torch.Size([88]), torch.Size([1, 4, 484]))

In [211]:
item[3]

'To specify more particularly one or two of the worst, it may be mentioned that in the Borough Compter'

In [ ]:
item

In [213]:
phone_prompt, audio_target, test_inp = multi_encode(
    item_phone_tokens,
    item_audio_tokens,
    n_phone_tokens=len(dataset.phone_dict),
    n_audio_tokens=1024,
    max_clip_length=5)
test_inp.shape

pre audio_tokens.shape torch.Size([1, 4, 484])
post audio_tokens.shape torch.Size([1500])
phone_tokens.shape: torch.Size([88])
audio_tokens.shape: torch.Size([1500])


torch.Size([1592])

### Training Process

In [1]:
from tqdm.notebook import tqdm

In [215]:
import torch.optim as optim

epochs = 10

MAX_LR       = 1e-2
# MAX_LR       = 1e-2
WEIGHT_DECAY = 1e-4
GRAD_CLIP    = 0.1

optimizer = optim.Adam(
    model.parameters(),
    lr=MAX_LR)
    #,weight_decay=WEIGHT_DECAY)

# def get_lr(optimizer):
#     for param_group in optimizer.param_groups:
#         return param_group['lr']

# sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, MAX_LR, epochs=epochs, 
#                                                 steps_per_epoch=len(trainloader))

In [216]:
test_inp.dtype

torch.int64

In [217]:
test_inp

tensor([1099, 1084, 1091,  ...,  859,  222, 1102], device='cuda:0')

In [218]:
test_inp.dtype

torch.int64

In [219]:
test_inp.shape

torch.Size([1592])

In [220]:
import torch.nn.functional as F

EPOCHS = 1000
PRINT_INTERVAL = 100

seq_len = 2048

def train(model, trainloader):
    model.train()
    
    padding_len = max(0, seq_len - test_inp.size(0))
    n_test_inp = F.pad(test_inp, (0, padding_len))
    batch = n_test_inp.unsqueeze(0)
    # print(batch.shape)
    loss = model(batch, return_loss=True)
    # loss = model(next(trainloader), return_loss=True)
    loss.backward()
    return loss

# pbar = tqdm.tqdm(EPOCHS, mininterval=10., desc='training')
for epoch in range(EPOCHS):
    loss = train(model, train_loader)
    optimizer.step()
    optimizer.zero_grad()
    mem_gb = get_reserved_mem_gb()
    if epoch % PRINT_INTERVAL == 0:
        print(f"Reserved Memory (GB): {mem_gb}, loss: {loss.item()}")
    #' pbar.set_description(f"Reserved Memory (GB): {mem_gb}, loss: {loss.item()}")

Reserved Memory (GB): 3.08984375, loss: 7.007445335388184
Reserved Memory (GB): 3.08984375, loss: 0.03312724456191063
Reserved Memory (GB): 3.08984375, loss: 0.016318222507834435
Reserved Memory (GB): 3.08984375, loss: 2.5148987770080566
Reserved Memory (GB): 3.08984375, loss: 0.025118699297308922
Reserved Memory (GB): 3.08984375, loss: 0.009873692877590656
Reserved Memory (GB): 3.08984375, loss: 0.007557244971394539
Reserved Memory (GB): 3.08984375, loss: 0.007007556967437267
Reserved Memory (GB): 3.08984375, loss: 0.006757958326488733
Reserved Memory (GB): 3.08984375, loss: 0.006608926225453615


### Evaluate

In [221]:
phone_prompt.shape

torch.Size([90])

In [227]:
audio_target

tensor([1101,   25,  574,  ...,  859,  222, 1102], device='cuda:0')

In [223]:
def generate(model, prompt):
    model.eval()

    prompt = prompt.unsqueeze(0)
    sample = model.generate(prompt)
    sample = sample.flatten(1)
    print("sample:", sample, sample.shape)

    return prompt, sample

prompt, sample = generate(model, phone_prompt)

100%|██████████| 1958/1958 [00:26<00:00, 74.08it/s]

sample: tensor([[1099, 1084, 1091,  ...,   43,  881,  784]], device='cuda:0') torch.Size([1, 2048])


In [224]:
sample.shape

torch.Size([1, 2048])

In [228]:
STT, ETT, SAT, EAT = [len(dataset.phone_dict) + 1024 + i
                          for i in range(4)]
sample.index(STT)

AttributeError: 'Tensor' object has no attribute 'index'

In [ ]:
out = generate_audio(
    sample,
    n_phone_tokens=len(dataset.phone_dict),
    n_audio_tokens=1024)

In [226]:
out

True